In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import os
import glob
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.models import Model
from sklearn import preprocessing
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image
from PIL import Image
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [20]:
def read_images_from_dir(directory):
    images = []
    for folder in folder_names:
        folder_path = os.path.join(directory, folder)
        print(folder_path)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            print(image_path)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not read image file {image_path}")
                continue
            print(image.shape)
            image = cv2.resize(image, (224, 224))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert from BGR to RGB
            images.append((image, int(folder)))  # Store the image and its class label (0 or 1) as a tuple
    return images

In [21]:
class_names = ['dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning' , 'rain', 'rainbow', 'rime', 'sandstorm', 'snow']
folder_names = ["0",     "1",      "2",     "3",    "4",      "5",        "6",      "7",      "8",      "9",      "10" ] 

In [ ]:
all_data = read_images_from_dir('dataset')

In [55]:
x = np.array([np.asarray(Image.fromarray(t[0])) for t in all_data])
y = np.array([t[1] for t in all_data])

In [59]:
print(len(x))
print(len(y))
print(x.shape)

6860
6860
(6860, 224, 224, 3)


In [60]:
from tensorflow.keras.applications.resnet50 import preprocess_input
i=0
for i in range(len(x)):
    x[i] = preprocess_input(x[i])
    i += 1
print(x.shape)


(6860, 224, 224, 3)


In [36]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)

train_features = x_train.reshape(x_train.shape[0], -1)
test_features = x_test.reshape(x_test.shape[0], -1)

print(x_train.shape, y_train.shape)

print(x_test.shape, y_test.shape)


clf = svm.SVC(kernel='linear', C=1).fit(train_features, y_train)
clf.score(test_features, y_test)

(4116, 224, 224, 3) (4116,)
(2744, 224, 224, 3) (2744,)


0.3651603498542274

In [68]:
def res50Built(Train, labelTrain, Test, labelTest, imageDim):
    epochs = 40
    num_classes = 11
    model = Sequential()
    pretrained_model = tf.keras.applications.ResNet50(include_top=False, 
                                                input_shape = (224, 224, 3),
                                                pooling='avg' , 
                                                classes=11,
                                                weights='imagenet')
    for layer in pretrained_model.layers:
      layer.trainable=False

    model.add(pretrained_model)
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        'sgd',
        loss='categorical_crossentropy',
        metrics=[tf.keras.metrics.AUC(name="accuracy"), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    model.fit(
        Train,
        to_categorical(labelTrain),
        epochs=epochs,
        validation_data=(Test, to_categorical(labelTest)),
    )

    model.summary()

    predictions = model.predict(Test)
    # print(predictions)
    predictions = np.argmax(predictions, axis=1)
    print('sum of labeltest: ', sum([labelTest[i] == predictions[i] for i in range(len(predictions))]))
    print('en of label test: ', len(labelTest))
    precision = precision_score(labelTest, predictions, average='micro')
    recall = recall_score(labelTest, predictions, average='micro')
    fscore = f1_score(labelTest, predictions, average='micro')
    print('precision: ', precision, 'recall: ', recall, "f1 score: ", fscore)
    # print(predictions)
    cm = confusion_matrix(labelTest, predictions)
    print('cm: ', cm)
    return predictions

In [ ]:
#splitting the dataset into train and test set by k-fold cross validation and training data using ResNet50
kf = KFold(n_splits=5, shuffle=True, random_state=0)  #5-fold cross validation 
fold = 1
accuracies = []
for train_index, test_index in kf.split(x):

    x_train, x_test = x[train_index], x[test_index]  #training and testing data 
    y_train, y_test = y[train_index], y[test_index]  #training and testing labels
    
    
    print(x_train.shape, x_test.shape)
    
    x_train2 = x_train.reshape(-1, 224, 224, 3)
    x_test2 = x_test.reshape(-1, 224, 224, 3)
    predictions = res50Built(x_train2, y_train, x_test2, y_test, 3)
    accuracy = accuracy_score(y_test, predictions, average='micro')
    print('Fold {}: accuracy = {:.2f}%'.format(fold, accuracy*100))
    
    # Add the accuracy to the list of accuracies
    accuracies.append(accuracy)
    
    fold += 1

# Print the mean and standard deviation of the accuracies

print('Mean accuracy = {:.2f}%'.format(np.mean(accuracies)*100))
print('Std deviation = {:.2f}%'.format(np.std(accuracies)*100))


(5488, 224, 224, 3) (1372, 224, 224, 3)
Epoch 1/40
172/172 [==============================] - 189s 1s/step - loss: 0.9333 - accuracy: 0.9553 - precision_5: 0.8306 - recall_5: 0.5556 - val_loss: 0.6100 - val_accuracy: 0.9805 - val_precision_5: 0.8619 - val_recall_5: 0.7187
Epoch 2/40
172/172 [==============================] - 187s 1s/step - loss: 0.5283 - accuracy: 0.9850 - precision_5: 0.8813 - recall_5: 0.7697 - val_loss: 0.5889 - val_accuracy: 0.9799 - val_precision_5: 0.8487 - val_recall_5: 0.7522
Epoch 3/40
172/172 [==============================] - 186s 1s/step - loss: 0.4364 - accuracy: 0.9898 - precision_5: 0.8981 - recall_5: 0.8110 - val_loss: 0.5169 - val_accuracy: 0.9848 - val_precision_5: 0.8787 - val_recall_5: 0.7813
Epoch 4/40
172/172 [==============================] - 187s 1s/step - loss: 0.3736 - accuracy: 0.9926 - precision_5: 0.9134 - recall_5: 0.8382 - val_loss: 0.4854 - val_accuracy: 0.9860 - val_precision_5: 0.8813 - val_recall_5: 0.8061
Epoch 5/40
172/172 [========

In [ ]:
#linear regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


for train_index, test_index in kf.split(x):

    x_train, x_test = x[train_index], x[test_index]  #training and testing data for independent variables 
    y_train, y_test = y[train_index], y[test_index]  #training and testing data for dependent variable
    
    train_features = x_train.reshape(x_train.shape[0], -1)
    test_features = x_test.reshape(x_test.shape[0], -1)
    #fitting linear regression to the training set    
    regressor = LinearRegression()   #creating an object of LinearRegression class 
    regressor.fit(train_features, y_train)  #fitting the linear regression model to our training set
    
    #predicting the test set results    
    y_pred = regressor.predict(test_features) 

In [40]:
y_pred

array([-7.14897808e+09, -2.66140052e+09,  1.22580323e+10, ...,
       -1.33950814e+10,  1.30502438e+10,  5.96462554e+09])